In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, HTML
import os
import re

notebook_path = os.path.abspath("adhoc.ipynb")
notebook_dir = str(os.path.dirname(notebook_path)).replace("\\\\", "/").replace("\\", "/")
output_folder = f"{notebook_dir}/../output"
raw_output = f"{output_folder}/raw"
curated_output = f"{output_folder}/curated"
semantic_output = f"{output_folder}/semantic"

nyse_tickers = f"{notebook_dir}/../wsb/nyse-listed.csv"
nasdaq_tickers = f"{notebook_dir}/../wsb/nasdaq-listed.csv"
curated_stockticker = f"{curated_output}/StockTicker.csv"

# output_html = f"{notebook_dir}/../index.html"

In [2]:
pd.set_option('display.max_colwidth', None)
sep = "|"

def convert_list(x):
    return x.strip("[]").split(", ")

df = pd.read_csv(curated_stockticker, sep=sep, 
converters={"title_ticker": lambda x: convert_list(x), 
"submission_text_ticker": lambda x: convert_list(x)}
)

df.dtypes
# display(df)
# display(df.iloc[[0]])

id                         object
Unnamed: 0                float64
title                      object
name                       object
upvote_ratio              float64
ups                         int64
score                       int64
sort                       object
created                    object
author                     object
num_comments                int64
flair                      object
permalink                  object
built_url                  object
url                        object
submission_text            object
last_updated               object
raw_filename               object
model                      object
title_regex                object
title_ticker               object
submission_text_regex      object
submission_text_ticker     object
dtype: object

In [3]:
df[df['id'].str.contains("l5gsp8")][["title","built_url","title_ticker", "submission_text_ticker"]]


,title,built_url,title_ticker,submission_text_ticker


In [4]:

def explode(df, cols):
        """method to explode list columns. mainly used for stock ticker columns

        :param df: pandas df
        :type df: obj
        :param cols: list of cols to explode
        :type cols: list
        """
        for col in cols:
            df = df.explode(col)

        return df

ticker_cols = ["title_ticker", "submission_text_ticker"]

exploded_df = explode(df, ["title_ticker", "submission_text_ticker"])
display(exploded_df)

,id,Unnamed: 0,title,name,upvote_ratio,ups,score,sort,created,author,...,built_url,url,submission_text,last_updated,raw_filename,model,title_regex,title_ticker,submission_text_regex,submission_text_ticker
0,l4kxzg,581.0,IM NOT SELLING THIS UNTIL AT LEAST $1000+ GME 🚀🚀🚀🚀🚀🚀 BUCKLE THE FUCK UP,t3_l4kxzg,0.89,28544,28544,hot,2021-01-25 13:06:32.000000,dumbledoreRothIRA,...,https://www.reddit.com/r/wallstreetbets/comments/l4kxzg/im_not_selling_this_until_at_least_1000_gme/,https://i.redd.it/u8ianxk1dgd61.png,NaN,2021-01-26 22:31:26.794724,../output/raw/2021/01/26/StockTicker_223143.csv,StockTicker,"['IM', 'NOT', 'LLING', 'THIS', 'UNTIL', 'AT', 'LEAST', 'GME', 'UCKLE', 'THE', 'FUCK']",GME,[],
1,l4lmrx,591.0,"GME Thoughts, YOLOs, Gains, Stonk Updates, 🚀🚀🚀: they all go here. Jan 25 edition.",t3_l4lmrx,0.86,17820,17820,hot,2021-01-25 13:56:35.000000,Dan_inKuwait,...,https://www.reddit.com/r/wallstreetbets/comments/l4lmrx/gme_thoughts_yolos_gains_stonk_updates_they_all/,https://www.reddit.com/r/wallstreetbets/comments/l4lmrx/gme_thoughts_yolos_gains_stonk_updates_they_all/,"Thanks all for the quick rise to max comments of 100k !!\r\n\r\nTo keep discussing the ways we can now financially shame our wives' boyfriends, continue the shitposting here:\r\n\r\nhttps://www.reddit.com/r/wallstreetbets/comments/l4syrd/gme_megathread_part_2/?utm_medium=android_app&utm_source=share",2021-01-26 22:31:26.794724,../output/raw/2021/01/26/StockTicker_223143.csv,StockTicker,['GME'],GME,[],
2,l4m4cc,576.0,$BB At The $GME Refuelling station 🚀🚀🚀,t3_l4m4cc,0.98,11485,11485,hot,2021-01-25 14:31:47.000000,Tradergurue,...,https://www.reddit.com/r/wallstreetbets/comments/l4m4cc/bb_at_the_gme_refuelling_station/,https://v.redd.it/4yy0qwt6sgd61,NaN,2021-01-26 22:31:26.794724,../output/raw/2021/01/26/StockTicker_223143.csv,StockTicker,"['$BB', '$GME']",GME,[],
3,l4nypg,532.0,PSA: Always understate your gains to friends and family,t3_l4nypg,0.93,15890,15890,hot,2021-01-25 16:35:15.000000,NE21,...,https://www.reddit.com/r/wallstreetbets/comments/l4nypg/psa_always_understate_your_gains_to_friends_and/,https://www.reddit.com/r/wallstreetbets/comments/l4nypg/psa_always_understate_your_gains_to_friends_and/,"Hello fellow astronauts,\r\n\r\nOn the eve of this great day, I think it's important to say that it's always best to understate/undersell your winnings to friends and family. I, for instance, will only tell when asked, and then give a figure around 50/60% of the real figure.\r\n\r\n\r\n**There are a multitude of reasons for doing this:**\r\n\r\n1. People do not understand the hard work and risk you have taken for these gains, and so will expect them for free\r\n\r\n2. Manipulative people will come out of the woodwork\r\n\r\n3. You may have taxes yet to pay (I know it sucks)\r\n\r\n4. You may lose all of this money yet (I am taking an investing break after this insane ride) \r\n\r\n5. The money is not a running tap, and is lump sums, so don't build a lifestyle around it \r\n\r\n6. Money is so much easier to spend than to make, please be sensible\r\n\r\nThat's all I have to say. Have a quiet cigar or champagne when you do cash out (above $1000 of course), or come up with some other excuse as to where the money came from. \r\n\r\n\r\nGME and BB to alpha centauri 🚀🚀🚀 Godspeed bois\r\n\r\nEdit: please also consider giving a portion of you funds to a charitable cause to make the world better in many ways",2021-01-26 22:31:26.794724,../output/raw/2021/01/26/StockTicker_223143.csv,StockTicker,[],,"['I', 'I', 'I', 'I', 'I', 'GME', 'BB']",GME
4,l4oh8d,560.0,Told my dad I’m going to use my miserable life saving to make money and be somewhat financially independent. He said “there’s always be home if something goes wrong but do it son try it make me proud” well I called him this morning and told him the retards and Reddit made me somewhat richer,t3_l4oh8d,0.96,7067,7067,hot,2021-01-25 17:05:19.000000,alexcthb9918,...,https://www.reddit.com/r/wallstreetbets/comments/l4oh8d/told_my

In [5]:
# https://stackoverflow.com/questions/23415500/pandas-plotting-a-stacked-bar-chart

# pd.pivot_table(exploded_df, values='id', index=ticker_cols, columns=['C'], aggfunc=np.sum)

title_df = exploded_df[["id", "title", "built_url", "title_ticker"]].drop_duplicates(subset=['id', 'title_ticker'])
title_df["title/submission"] = "title"
title_df = title_df.rename({"title_ticker": "ticker"},axis=1)

# title_df.groupby("title_ticker").nunique()
# agg_title = title_df[["id", "ticker", "title/submission"]].groupby(["ticker", "title/submission"]).agg(['count']).reset_index()

submission_df = exploded_df[["id", "title", "built_url", "submission_text_ticker"]].drop_duplicates(subset=['id','submission_text_ticker'])
submission_df["title/submission"] = "submission"
submission_df = submission_df.rename({"submission_text_ticker": "ticker"}, axis=1)

# agg_submission = submission_df[["id", "ticker", "title/submission"]].groupby(["ticker", "title/submission"]).agg(['count']).reset_index()


# agg_title.merge(agg_submission, on="ticker", how="outer")
# pd.concat([agg_submission,agg_title], ignore_index=True)
plot_df = pd.concat([title_df, submission_df], ignore_index=True)


# concat_df = concat_df[["ticker", "title/submission"]]

def filter_count(df, col, min):
    return df[df.groupby(col)[col].transform('count') > min]

plot_df = filter_count(plot_df, "ticker", 11)
# plot_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()
# .plot(kind='bar', stacked=True).figure.savefig(f'{notebook_dir}/StockTicker.png')



In [7]:
# drill_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()

# for row in drill_df.itertuples():
#     temp_data = [
#         ["submission_text", row.submission],
#         ["title", row.title]
#         ]
#     # print(f"{row.ticker}: {row.submission}")
#     # print(f"{row.ticker}: {row.title}")
#     H.add_drilldown_data_set(temp_data, 'column', row.ticker, name=row.ticker )

In [8]:
from highcharts import Highchart
H = Highchart(width=850, height=400)

"""
Drilldown chart can be created using add_drilldown_data_set method: 
add_drilldown_data_set(data, series_type, id, **kwargs):
id is the drilldown parameter in upperlevel dataset (Ex. drilldown parameters in data)
drilldown dataset is constructed similar to dataset for other chart
"""

data = [{
            'name': "Microsoft Internet Explorer",
            'y': 56.33,
            'drilldown': "Microsoft Internet Explorer"
        }, {
            'name': "Chrome",
            'y': 24.030000000000005,
            'drilldown': "Chrome"
        }, {
            'name': "Firefox",
            'y': 10.38,
            'drilldown': "Firefox"
        }, {
            'name': "Safari",
            'y': 4.77,
            'drilldown': "Safari"
        }, {
            'name': "Opera",
            'y': 0.9100000000000001,
            'drilldown': "Opera"
        }, {
            'name': "Proprietary or Undetectable",
            'y': 0.2,
            'drilldown': None
        }]

data_1 = [
    ["v11.0", 24.13],
    ["v8.0", 17.2],
    ["v9.0", 8.11],
    ["v10.0", 5.33],
    ["v6.0", 1.06],
    ["v7.0", 0.5]
]

data_2 = [
    ["v40.0", 5],
    ["v41.0", 4.32],
    ["v42.0", 3.68],
    ["v39.0", 2.96],
    ["v36.0", 2.53],
    ["v43.0", 1.45],
    ["v31.0", 1.24],
    ["v35.0", 0.85],
    ["v38.0", 0.6],
    ["v32.0", 0.55],
    ["v37.0", 0.38],
    ["v33.0", 0.19],
    ["v34.0", 0.14],
    ["v30.0", 0.14]      
]

data_3 = [
    ["v35", 2.76],
    ["v36", 2.32],
    ["v37", 2.31],
    ["v34", 1.27],
    ["v38", 1.02],
    ["v31", 0.33],
    ["v33", 0.22],
    ["v32", 0.15]
]

data_4 = [
    ["v8.0", 2.56],
    ["v7.1", 0.77],
    ["v5.1", 0.42],
    ["v5.0", 0.3],
    ["v6.1", 0.29],
    ["v7.0", 0.26],
    ["v6.2", 0.17]
]

data_5 = [
    ["v12.x", 0.34],
    ["v28", 0.24],
    ["v27", 0.17],
    ["v29", 0.16]
]

options = {
    'chart': {
        'type': 'column'
    },
    'title': {
        'text': 'Browser market shares. January, 2015 to May, 2015'
    },
    'subtitle': {
        'text': 'Click the columns to view versions. Source: <a href="http://netmarketshare.com">netmarketshare.com</a>.'
    },
    'xAxis': {
        'type': 'category'
    },
    'yAxis': {
        'title': {
            'text': 'Total percent market share'
        }

    },
    'legend': {
        'enabled': False
    },
    'plotOptions': {
        'series': {
            'borderWidth': 0,
            'dataLabels': {
                'enabled': True,
                'format': '{point.y:.1f}%'
            }
        }
    },

    'tooltip': {
        'headerFormat': '<span style="font-size:11px">{series.name}</span><br>',
        'pointFormat': '<span style="color:{point.color}">{point.name}</span>: <b>{point.y:.2f}%</b> of total<br/>'
    }, 
    
}
   
H.set_dict_options(options)

H.add_data_set(data, 'column', "Brands", colorByPoint= True)
H.add_drilldown_data_set(data_1, 'column', 'Microsoft Internet Explorer', name='Microsoft Internet Explorer' )
H.add_drilldown_data_set(data_2, 'column', 'Chrome', name='Chrome')
H.add_drilldown_data_set(data_3, 'column', 'Firefox', name='Firefox')
H.add_drilldown_data_set(data_4, 'column', 'Safari', name='Safari')
H.add_drilldown_data_set(data_5, 'column', 'Opera', name='Opera')

H.save_file(f"{notebook_dir}/../sample_drilldown")

In [9]:
from highcharts import Highchart
from pandas.core.series import Series

H = Highchart(width=850, height=600)

# chart.add_data_set(data, series_type='line', name='Example Series')
agg_df = plot_df[["ticker", "id"]].groupby(["ticker"]).nunique().reset_index()
# .count().fillna(0)

# if not isinstance(agg_df, Series):
#     raise TypeError("agg_df not pandas.core.series.Series")

# display(agg_df)
agg_df["drilldown"] = agg_df["ticker"]
data = agg_df.rename({"id": "y", "ticker": "name"}, axis=1).to_dict('records')

options = {
    'chart': {
        'type': 'column'
    },
    'title': {
        'text': 'Stock Tickers mentioned on wallsteetbets'
    },
    'subtitle': {
        'text': 'Click the columns to view breakdown. Source: <a href="https://www.reddit.com/r/wallstreetbets/">wallstreetbets on Reddit</a>.'
    },
    'xAxis': {
        'type': 'category'
    },
    'yAxis': {
        'title': {
            'text': 'Number of Mentions'
        }

    },
    'legend': {
        'enabled': False
    },
    'plotOptions': {
        'series': {
            'borderWidth': 0,
            'dataLabels': {
                'enabled': True,
                'format': '{point.y:.1f}'
            }
        }
    },

    'tooltip': {
        'headerFormat': '<span style="font-size:11px">{series.name}</span><br>',
        'pointFormat': '<span style="color:{point.color}">{point.name}</span>: <b>{point.y:.2f}</b> mentions<br/>'
    }, 
    
}

H.set_dict_options(options)

H.add_data_set(data, 'column', "Stock Tickers", colorByPoint= True)

drill_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()

for row in drill_df.itertuples():
    temp_data = [
        ["submission_text", row.submission],
        ["title", row.title]
        ]
    H.add_drilldown_data_set(temp_data, 'column', row.ticker.strip(), name=row.ticker.strip() )


H.save_file(f"{notebook_dir}/../drilldown")

data

[{'name': '', 'y': 752, 'drilldown': ''},
 {'name': 'A', 'y': 19, 'drilldown': 'A'},
 {'name': 'DD', 'y': 20, 'drilldown': 'DD'},
 {'name': 'GME', 'y': 226, 'drilldown': 'GME'},
 {'name': 'I', 'y': 162, 'drilldown': 'I'},
 {'name': 'K', 'y': 13, 'drilldown': 'K'},
 {'name': 'NOK', 'y': 22, 'drilldown': 'NOK'}]

In [10]:
stack_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()
stack_df

title/submission,ticker,submission,title
0,,642,453
1,A,10,9
2,DD,7,13
3,GME,64,183
4,I,88,77
5,K,3,10
6,NOK,1,21


In [11]:
for row in stack_df.itertuples():
    print(f"{row.ticker}: {row.submission}")
    print(f"{row.ticker}: {row.title}")

: 642
: 453
A: 10
A: 9
DD: 7
DD: 13
GME: 64
GME: 183
I: 88
I: 77
K: 3
K: 10
NOK: 1
NOK: 21


In [12]:
# -*- coding: utf-8 -*-
"""
Highcharts Demos
Stacked and grouped column: http://www.highcharts.com/demo/column-stacked-and-grouped
"""
from highcharts import Highchart

# H = Highchart()
H = Highchart(width=750, height=600)

# data1 = [5, 3, 4, 7, 2]
# data2 = [3, 4, 4, 2, 5]
# data3 = [2, 5, 6, 2, 1]
# data4 = [3, 0, 4, 4, 3]

options = {
	'title': {
        'text': 'Stock Ticker mentions on wallstreetbets'
    },

    'xAxis': {
        # 'categories': ['Apples', 'Oranges', 'Pears', 'Grapes', 'Bananas']
        'categories': stack_df["ticker"].tolist()
    },

    'yAxis': {
        'allowDecimals': False,
        'min': 0,
        'title': {
            'text': 'Number of Mentions'
        }
    },

    'tooltip': {
        'formatter': "function () {\
                        return '<b>' + this.x + '</b><br/>' +\
                            this.series.name + ': ' + this.y + '<br/>' +\
                            'Total: ' + this.point.stackTotal;\
                    }"
    },
    'plotOptions': {
        'column': {
            'stacking': 'normal'
        }
    }
}
H.set_dict_options(options)

H.add_data_set(stack_df["submission"].tolist(), "column", "submission text")
H.add_data_set(stack_df["title"].tolist(), "column", "title of submission")

# H.add_data_set(data1, 'column', 'John', stack='male' )
# H.add_data_set(data2, 'column', 'Joe', stack='male')
# H.add_data_set(data3, 'column', 'Jane', stack='female')
# H.add_data_set(data4, 'column', 'Janet', stack='female')

# H.htmlcontent
H.save_file(f"{notebook_dir}/../columnstack")